# Instructions
- Don't use "-" in comments
- Only use / for dates, or Name date format
- Use - to seperate time
- Don't use dash (-) in lines other than to seperate times

In [ ]:
!python generate_timesheet.py \
    --file 2022/March.txt \
    --rate 25 \
    --discounted_rate 25 \
    --threshold 14

# Detailed Exploratory Notebook

## Tips
- Only use / for dates, or Name date format
- Use - to seperate time
- Don't use dash (-) in lines other than to seperate times

In [ ]:
from datetime import datetime, timedelta
from dateutil.parser import parse
import pandas as pd

In [ ]:
file_name = "2022/March.txt"

In [ ]:
time_zone = 5.5
hour_difference = 2.5
hour_sign = '+'

base_rate = 25
discount_threshold = 14 # in hours
discounted_rate = 20

In [ ]:
f = open(file_name, "r")
txt_data = f.read()
print(txt_data)

In [ ]:
# sample_date = '8th April'
# print(parse(sample_date, dayfirst=True).date())
# sample_date = '08/05'
# print(parse(sample_date, dayfirst=True).date())

In [ ]:
def stringtodate(string):
    datetime_object = parse(string, dayfirst=True).date()
    return datetime_object

def stringtotime(time, date):
    date = stringtodate(date)
    time = datetime.strptime(time, '%H:%M').time()
    date_time = datetime.combine(date, time)
    return date_time

def days_hours_minutes(td):
    return td.total_seconds() / 60

In [ ]:
data = pd.DataFrame(columns = ["Date","Start Time", "End Time", "Comment"])

with open(file_name, "r") as f:
    lines = f.readlines()
    
    for line in lines:
        if ':' not in line:
            date = line.rstrip('\n').strip()
        elif line == '\n':
            continue
        else:
            start_time, end_time = line.split('-')
            start_time, end_time = start_time.strip(), end_time.strip()
            
            if '(' in end_time:
                end_time, comment = end_time.split('(')
                end_time = end_time.strip()
                comment = comment.strip('()')
            else:
                end_time, comment = end_time, None

            row = {"Date":date, "Start Time":stringtotime(start_time, date), "End Time":stringtotime(end_time, date), "Comment":comment}
            row = pd.DataFrame(row, index=[0])
            data = pd.concat([data, row], axis=0)

data['Date'] = data['Date'].apply(lambda x:parse(x, dayfirst=True).date())
data = data.sort_values(['Date'])
data['Total Time'] = data['End Time'] - data['Start Time']
data['Total Minutes'] = data['Total Time'].apply(days_hours_minutes)
data['Client Start Time'] = data['Start Time'] + pd.to_timedelta(hour_difference, unit='h')
data['Client End Time'] = data['End Time'] + pd.to_timedelta(hour_difference, unit='h')

data = data[data.Comment != 'break']

data['Start Time'] = data['Start Time'].dt.strftime('%H:%M')
data['End Time'] = data['End Time'].dt.strftime('%H:%M')
data['Client Start Time'] = data['Client Start Time'].dt.strftime('%H:%M')
data['Client End Time'] = data['Client End Time'].dt.strftime('%H:%M')
print(data)

In [ ]:
data

In [ ]:
total_minutes_by_date = data.groupby(by='Date', as_index=False).sum() # data[data['Comment']!='call']
total_minutes_by_date

In [ ]:
print('Total time in minutes: {}\nTotal time in days: {}.'.format(data['Total Minutes'].sum(), data['Total Time'].sum()))

In [ ]:
all_dates = sorted(list(set(row.Date for index, row in data.iterrows())))
all_dates

In [ ]:
data[['Date','Total Minutes']]

In [ ]:
total_minutes_by_date['Total Minutes'].sum()

In [ ]:
def main():
    print("Invoicely\nTime in GMT {}{}\n".format(hour_sign,hour_difference+time_zone))

    print("{} ({} to {})".format(file_name.split('.txt')[0], all_dates[0], all_dates[-1]))

    print('Actual Total time for the week:- {} mins or {:.2f} hours'.format(int(data['Total Minutes'].sum()), \
                                                  int(data['Total Minutes'].sum())/60))
    # print('\n14-hour week discount calculations:- \nP.S:- It excludes solo time, as it\'s already charged at $20/hr\n\n')
    if total_minutes_by_date['Total Minutes'].sum() > discount_threshold*60:
        print("Total time greater than {} hrs, hence billing at ${}/hr.".format(discount_threshold, discounted_rate))
    else:
        print("Total time below 14 hrs, hence billing at ${}/hr".format(base_rate))
    print('#'*50)

    for index, (date, tm) in total_minutes_by_date.iterrows():
        print('\n{}'.format(date))
        print('Actual Total time:- {} mins. or {:.2f} hours.'.format(int(tm), tm/60))
        print('Discounted Total time:- [] mins or [] hours.\n')
        current_date = data[data['Date']==date]
        print("Breakdown")
        for index, row in current_date.iterrows():
            if row['Comment'] == None : # or row['Comment'].strip() == 'call'
                print('{} - {} ({} mins)'\
                      .format(row['Client Start Time'], 
                              row['Client End Time'], 
                              int(row['Total Minutes'])))
            else:
                print('{} - {} ({} mins)'\
                      .format(row['Client Start Time'], 
                              row['Client End Time'], 
                              int(row['Total Minutes'])), 
                      end=' ({})\n'.format(row['Comment']))
    #         if row['Comment'] != None:
    #             print('({})\n'.format(row['Comment']))
        print('\n'+'#'*50)

In [ ]:
main()